In [ ]:
import itertools
import os
import time
from tempfile import TemporaryDirectory
from typing import Literal, Optional, TypeAlias, TypedDict

import cv2
import cv2.typing as cvt
import numpy as np
from PIL import Image
from tqdm import tqdm

"""
Helpers
"""

ImageKind: TypeAlias = Literal["file", "rand"]


class TestImage(TypedDict):
    kind: ImageKind
    size: tuple[int, int]
    pil_image: Image.Image
    cv2_image: cvt.MatLike
    path: Optional[str]


class LibraryStats(TypedDict):
    avg_time: float
    avg_time_std: float
    avg_filesize: float


class TestCaseResult(TypedDict):
    kind: ImageKind
    size: tuple[int, int]
    compress_level: int
    pil: LibraryStats
    cv2: LibraryStats
    path: Optional[str]


def get_images_from_path(path: str) -> TestImage:
    pil_image = Image.open(path)
    cv2_image = cv2.imread(path)
    return TestImage(
        kind="file",
        size=pil_image.size,
        pil_image=pil_image,
        cv2_image=cv2_image,
        path=path,
    )


def get_images_from_size(size: tuple[int, int]) -> TestImage:
    np_img = np.random.randint(low=0, high=255, size=size, dtype=np.uint8)
    pil_image = Image.fromarray(np_img)
    cv2_image = cv2.cvtColor(np_img, cv2.COLOR_RGB2BGR)
    return TestImage(
        kind="rand",
        size=size,
        pil_image=pil_image,
        cv2_image=cv2_image,
        path=None,
    )


def save_pil(img: Image.Image, path: str, compress_level: int):
    img.save(path, "PNG", compress_level=compress_level)


def save_cv2(img: cvt.MatLike, path: str, compress_level: int):
    cv2.imwrite(
        path,
        img,
        [cv2.IMWRITE_PNG_COMPRESSION, compress_level],
    )


def run_test(test_cases: list[tuple[TestImage, int]], iterations: int) -> list[TestCaseResult]:
    results: list[TestCaseResult] = []
    with TemporaryDirectory() as tmpdir:
        for test_image, compress_level in tqdm(test_cases):
            test_case_result = TestCaseResult(
                kind=test_image["kind"],
                size=test_image["size"],
                compress_level=compress_level,
                path=test_image["path"],
                pil=LibraryStats(
                    avg_filesize=0,
                    avg_time=0,
                    avg_time_std=0,
                ),
                cv2=LibraryStats(
                    avg_filesize=0,
                    avg_time=0,
                    avg_time_std=0,
                ),
            )
            for lib in ["pil", "cv2"]:
                times = []
                filesizes = []
                for i in range(-1, iterations):
                    img_path = (
                        f"{tmpdir}/{lib}_{test_image['size']}_{compress_level}_{i}.png"
                    )
                    start = time.time()
                    if lib == "pil":
                        save_pil(test_image["pil_image"], img_path, compress_level)
                    else:
                        save_cv2(test_image["cv2_image"], img_path, compress_level)
                    end = time.time()
                    if i == -1:
                        # warmup
                        continue
                    times.append((end - start) * 1000)
                    filesizes.append(os.path.getsize(img_path))
                test_case_result[lib]["avg_time"] = float(np.average(times))
                test_case_result[lib]["avg_time_std"] = float(np.std(times))
                test_case_result[lib]["avg_filesize"] = float(
                    np.average(filesizes) / 1024
                )
            results.append(test_case_result)
    return results

In [ ]:
"""
Test Cases
"""

# compression level - both PIL and cv2 use 0 (disabled) to 9 (max compression) scales, but it's unclear if they are the same
compress_levels = range(0, 10)

# sizes used for pure random noise images
sizes = [
    (512, 512),
    (512, 768),
    (768, 768),
    (768, 1024),
    (1024, 1024),
    (1024, 1536),
    (1536, 1536),
    (1536, 2048),
    (2048, 2048),
]

# real images
image_paths = [
    "./512x512.png",
    "./512x768.png",
    "./768x768.png",
    "./768x1024.png",
    "./1024x1024.png",
    "./1024x1536.png",
    "./1536x1536.png",
    "./1536x2048.png",
    "./2048x2048.png",
]

images_from_file = [get_images_from_path(path) for path in image_paths]
images_from_rand = [get_images_from_size(size) for size in sizes]

test_images = (
    images_from_file  # + images_from_rand # uncomment to test random noise images
)

test_cases = list(itertools.product(test_images, compress_levels))

results = run_test(test_cases, iterations=5)

In [ ]:
"""
Results!
"""

green = (0, 255, 0)
red = (255, 0, 0)
cyan = (0, 255, 255)


def colored(r, g, b, text):
    return f"\033[38;2;{r};{g};{b}m{text}\033[0m"


pil_win_count = 0
cv2_win_count = 0
for result in results:
    winner = "pil" if result["pil"]["avg_time"] < result["cv2"]["avg_time"] else "cv2"
    t_delta = (
        result["pil"]["avg_time"] - result["cv2"]["avg_time"]
        if winner == "pil"
        else result["cv2"]["avg_time"] - result["pil"]["avg_time"]
    )

    print(
        colored(
            *cyan,
            f"{result['kind']} @ {result['size']}, compress_level={result['compress_level']} -> {winner} wins by {t_delta:.2f}ms",
        )
    )

    if winner == "pil":
        colors = dict(pil=green, cv2=red)
        pil_win_count += 1
    else:
        cv2_win_count += 1
        colors = dict(pil=red, cv2=green)

    for lib in ["pil", "cv2"]:
        avg_time = f"{result[lib]['avg_time']:.2f}ms"
        avg_time_std = f"{result[lib]['avg_time_std']:.2f}ms"
        avg_filesize = f"{result[lib]['avg_filesize']:.2f}kb"
        print(
            colored(
                *colors[lib],
                f"{lib}: {avg_time} +-{avg_time_std} ({avg_filesize})",
            )
        )

    print()

print(f"pil wins: {pil_win_count}")
print(f"cv2 wins: {cv2_win_count}")